In [1]:
import numpy as np
import pandas as pd
import timeit
import json
import re
import itertools
import matplotlib.pyplot as plt
#import cupy
from sklearn import linear_model

import spacy
import en_trf_xlnetbasecased_lg
from spacy.pipeline import EntityRuler

In [3]:
yelp_business_datapath = '/home/matt_valley/PycharmProjects/insight_2020a_project/Resto_names/yelp_dataset/business.json'

num_entries = 100000
users = []
with open(yelp_business_datapath) as fl:
    for i, line in enumerate(fl):
        users.append(json.loads(line))
        if i+1 >= num_entries:
            break
df = pd.DataFrame(users)
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [4]:
print(df.keys())

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')


In [5]:
# get restaurants and their reviews

re_obj = re.compile('Restaurant*')
def filter_categories(row):
    if row:
        if re_obj.search(row):
            return True
        else:
            return False
    else:
        return False
    
resto_df = df[df.categories.apply(lambda x: filter_categories(x))]
print(len(resto_df))
print(len(df))

30776
100000


In [6]:
# load spacy model

#model = 'en_core_web_sm' # for testing on laptop
model = 'en_vectors_web_lg' # many more words
nlp = spacy.load(model)

In [7]:
#Get embeddings for all resto names

def embed_name(name):
    spacy_obj = nlp(name)
    return spacy_obj.vector

test_name = resto_df['name'].iloc[0]
print(test_name)

vec = embed_name(test_name)
print(vec.shape)

Emerald Chinese Restaurant
(300,)


In [8]:
resto_df['name_vector'] = resto_df.name.apply(lambda x: embed_name(x))

/home/matt_valley/anaconda3/envs/spacy/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
resto_df.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,name_vector
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","[0.015270342, 0.091443665, 0.17911, -0.353965,..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...","[0.19946666, 0.015730664, 0.11467668, -0.44225..."
11,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...","[-0.22609499, 0.03694001, 0.25902498, 0.206840..."
13,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,16,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...","[0.024047563, 0.15770334, 0.16039033, 0.045059..."
17,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119,36.100016,-115.128529,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Restaurants, Italian",None,"[0.39877, 0.26875252, -0.12409, -0.3414675, 0...."


In [10]:
lin_model = linear_model.LinearRegression(normalize=True)

y = resto_df.stars.values
#y = np.expand_dims(y, axis=1)
print(y.shape)

X = np.empty([len(resto_df), vec.shape[0]])
for r in range(len(resto_df)):
    X[r,:] = resto_df['name_vector'].iloc[r]
print(X.shape)

(30776,)
(30776, 300)


In [11]:
lin_model.fit(X,y)
print(lin_model.coef_)
print(lin_model.score(X,y))

[-1.03017210e-01 -9.78880845e-02  3.08338408e-02  7.39238901e-02
  1.69803487e-02 -1.74552968e-02 -5.88858612e-02  6.06968570e-02
 -7.33528890e-03 -6.53198747e-02 -8.15935691e-02  7.78375812e-02
  1.49188579e-03 -1.93998041e-02  9.68267789e-03 -2.17889983e-02
 -6.59998300e-02 -1.85298685e-03 -5.02905001e-02  4.51310361e-02
 -3.89208300e-02 -1.10566853e-01 -5.83299883e-02 -6.80658512e-03
  6.06558363e-02  1.27308365e-01  1.56253261e-01  2.71355378e-02
 -1.47905311e-01 -5.56006442e-02  3.31255158e-02 -5.50236923e-02
 -6.21124492e-02  2.56634133e-02  2.54921563e-02  5.11189129e-02
  9.62924430e-02 -7.76077726e-03  2.64191513e-03  2.62338028e-02
  3.39456075e-02 -2.93988670e-02  2.19748553e-01  1.33804377e-01
 -2.82846815e-03  1.01785673e-01  5.13908340e-02 -1.71665973e-01
  3.04764935e-02 -1.18363680e-01  4.14495708e-03  3.24971804e-02
 -1.05968607e-01  8.02602890e-03 -1.26553860e-02 -5.91050746e-02
  6.21957660e-02 -3.62500302e-02 -7.32126615e-02  7.48318267e-02
  3.98282932e-02 -4.79204

In [42]:
# NOW PARSE REVIEWS USING SENTIMENT ANALYSIS

yelp_review_datapath = '/home/matt_valley/PycharmProjects/insight_2020a_project/Resto_names/yelp_dataset/review.json'

reviews = []
with open(yelp_review_datapath) as fl:
    for i, line in enumerate(fl):
        reviews.append(json.loads(line))
        if i+1 >= num_entries:
            break
df_rev = pd.DataFrame(reviews)
df_rev.head()


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38
